In [ ]:
using LinearAlgebra
using Plots
using Distributions

In [ ]:

function multivariate_normal(x, μ, Σ)
    Det = det(Σ)
    Inv = inv(Σ)
    d = length(x)
    z = 1 / √((2π)^d * Det)
    y = z * exp(-(x .- μ)' * Inv * (x .- μ) / 2.0)
    return y
end

function gmm(x, ϕs, μs, Σs)
    K = length(ϕs)
    y = 0
    
    for k in range(1,K)
        ϕ, μ, Σ = ϕs[k], μs[k,:], Σs[k]
        y += ϕ * multivariate_normal(x, μ, Σ)
    end
    return y
end


function likelihood(xs, ϕs, μs, Σs)
    eps = 1e-8
    L = 0
    N = size(xs)[1]
    
    for n in range(1,N)
        y = gmm(xs[n,:], ϕs, μs, Σs)
        L += log(y + eps)
    end
    return L/N
end


function EStep(q, xs, ϕs, μs, Σs)
    N = size(xs)[1]
    K = length(ϕs)
    
    for n in range(1,N)        
        for k in range(1,K)
            q[n,k] = ϕs[k] * multivariate_normal(xs[n,:], μs[k,:], Σs[k])
        end
        q[n,:] ./= gmm(xs[n,:], ϕs, μs, Σs) 
    end            
    return q

end

function MStep(q, xs, ϕs, μs, Σs)
    N = size(xs)[1]
    K = length(ϕs)
    q_sum = sum(q,dims=1)

    for k in range(1,K)

        ϕs[k] = q_sum[k] / size(q)[1]
        
        μs[k,:] = sum(q[:,k].*xs, dims=1) ./ q_sum[k]

        cnt = zeros(size(Σs[k]))
        for n in range(1,N)
            z = xs[n,:] - μs[k,:]    
            cnt += q[n,k] .* z * z'
        end
        
        Σs[k] = cnt ./ q_sum[k]
        
    end
    return ϕs,μs,Σs
    
end




In [ ]:
iters = 10000

#混合率
ϕs = [0.5 0.5]
K = length(ϕs)
#平均
μs = [0.0 50.0; 0.0 100.0]
#分散共分散行列
Σs = [I(K)+zeros(BigFloat,K,K), I(K)+zeros(BigFloat,K,K)]

#更新の閾値
θ = 1e-4

#尤度
current_likelihood = likelihood(xs, ϕs, μs, Σs)


q = zeros(BigFloat,size(xs)[1],length(ϕs))



for i in range(1,iters)
    
    println(current_likelihood)
    
    q = EStep(q, xs, ϕs, μs, Σs)

    ϕs, μs, Σs = MStep(q, xs, ϕs, μs, Σs)
    
    next_likelihood = likelihood(xs, ϕs, μs, Σs)
    
    diff = abs(next_likelihood - current_likelihood)
    if diff < θ
        break
    end
    
    
    current_likelihood = next_likelihood
    
end

In [ ]:
#データ生成フェーズ

#作成するデータ数
Make_N = 500

new_xs = zeros(Make_N,size(xs)[2])

for n in range(1,Make_N)
    k = rand(Categorical(ϕs[:]))
    μ,Σ = μs[k,:],Σs[k]
    new_xs[n,:] = rand(MultivariateNormal(Float64.(μs[k,:]),Float64.(round.(Σs[k],digits=4))))

end